In [1]:
import urllib
import urllib.request
import json
import time
import numpy as np
from csv import writer
from datetime import datetime
import pandas as pd

In [2]:
#urllib.request.urlopen('https://api.thingspeak.com/update?api_key=R7I8AQE9FRJBYGN9&field1='+str('adsfsf'))

In [3]:
# a=urllib.request.urlopen('https://api.thingspeak.com/channels/508740/feeds.json?results=1')
# print(a)

In [4]:
# c= a.read()
# print(c)

In [5]:
a=urllib.request.urlopen('https://api.thingspeak.com/channels/508740/feeds.json?results=1').read()
res_dict= json.loads(a.decode('utf-8'))

In [6]:
mob1= res_dict['channel']['field1']

In [7]:
mob1

'mobile1'

In [8]:
mobile1 = res_dict['channel']['field1']
mobile1

'mobile1'

In [9]:
mobile1_values = res_dict['feeds'][0]['field1']
mobile1_values

'1106216.34354_3_0.140_3.322_9.358'

In [10]:
x=list(map(float, mobile1_values.split('_')))
x

[1106216.34354, 3.0, 0.14, 3.322, 9.358]

In [11]:
i=0
for item in x:  
    i=i+1
    if item==3.0:
        acceleros=[x[i],x[i+1],x[i+2]]
        sqrt_acceler = np.sqrt(sum(j*j for j in acceleros))
        print(mobile1,"accel",sqrt_acceler)

    if item==4.0:
        gyros=[x[i],x[i+1],x[i+2]]
        sqrt_gyro = np.sqrt(sum(j*j for j in gyros))
        print(mobile1,"Gyro:",sqrt_gyro)


mobile1 accel 9.93113528253442


In [12]:
head_cols = ['mobileno', 'dateandtime', 'acc', 'gyro','weight', 'height','gender', 'age']

In [13]:
def get_sqrt(mobileno, mobile_values):
    sqrt_acceler = 0
    sqrt_gyro = 0
    
    x=list(map(float, mobile_values.split('_')))
    i=0
    for items in x:  
        i=i+1
        if items==3.0:
            acceleros=[x[i],x[i+1],x[i+2]]
            sqrt_acceler = np.sqrt(sum(j*j for j in acceleros))
#             print(mobileno,"accel",sqrt_acceler)
            i=0
            
        if items==4.0:
            gyros=[x[i],x[i+1],x[i+2]]
            sqrt_gyro = np.sqrt(sum(j*j for j in gyros))
#             print(mobileno,"Gyro:",sqrt_gyro)
            i=0

    return sqrt_acceler, sqrt_gyro 

In [14]:
def read_data():
    # Current date time in local system
    dateTimeObj = datetime.now()
    now = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")
    
    weight1 = 65
    weight2 = 80
    
    height1 = 170
    height2 = 185
    
    gender1 = 'M'
    gender2 = 'M'
    
    age1 = 25
    age2 = 27
    
    a=urllib.request.urlopen('https://api.thingspeak.com/channels/508740/feeds.json?results=1').read()
    res_dict= json.loads(a.decode('utf-8'))
    print("Receiving data from url")
    mobile1 = res_dict['channel']['field1']
    mobile1_values = res_dict['feeds'][0]['field1']
    sqrt_acceler1, sqrt_gyro1 = get_sqrt(mobile1, mobile1_values)
#     print("got sqrt of accelero:",sqrt_acceler1," and gyro:", sqrt_gyro1,"from ",mobile1)
    
    sqrt_gyro2 = 0
    sqrt_acceler2 = 0
    mobile2 = res_dict['channel']['field2'] 
    mobile2_values = res_dict['feeds'][0]['field2']
    
    if mobile2_values is not None:
        sqrt_gyro2, sqrt_acceler2 = get_sqrt(mobile2, mobile2_values)
#         print("got sqrt of accelero:",sqrt_acceler2," and gyro:", sqrt_gyro2,"from ",mobile2)
        return sqrt_gyro1, sqrt_acceler1, sqrt_gyro2, sqrt_acceler2
    
    data1 = [mobile1, now, sqrt_acceler1, sqrt_gyro1, weight1, height1, gender1, age1]
    data2 = [mobile2, now, sqrt_acceler2, sqrt_gyro2, weight2, height2, gender2, age2]
    return data1, data2

In [15]:
# df = pd.DataFrame()

In [16]:
i=0
while True:
#     time.sleep(2)
    i=i+1
    data1, data2 = read_data()
    print("Data 1:",data1)
    print("Data 2:",data2)
#     with open('/home/jovyan/work/mldb_proj//data/file{}.csv'.format(i)) as f_object:
#             writer_object = writer(f_object)
# #             data = receive_data()
#             # Pass the list as an argument into
#             # the writerow()
#             writer_object.writerow(data1)
#             writer_object.writerow(data2)
#             #Close the file object
#             f_object.close()
    data =[data1, data2]
    df = pd.DataFrame(data,columns=head_cols)
    df.to_csv('/home/jovyan/work/mldb_proj//data/test{}.csv'.format(i), index =False)
    
#     df = pd.DataFrame(data2, index=False)
#     df.to_csv('/home/jovyan/work/mldb_proj//data/data2test{}.csv'.format(i))

    
#     print("Data added to data1test{}.csv".format(i))
#     print("Data added to data2test{}.csv".format(i))

Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:42:43.708718)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:42:43.708718)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:42:44.808132)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:42:44.808132)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:42:45.857914)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:42:45.857914)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:42:46.951880)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:42:46.951880)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:42:48.003070)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:42:48.003070)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['m

Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:43:40.012888)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:43:40.012888)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:43:41.083585)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:43:41.083585)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:43:42.169965)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:43:42.169965)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:43:43.279202)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:43:43.279202)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:43:44.576155)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:43:44.576155)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['m

Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:44:38.450416)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:44:38.450416)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:44:39.541991)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:44:39.541991)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:44:45.601745)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:44:45.601745)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:44:51.662395)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:44:51.662395)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['mobile1', '12-Jun-2021 (19:44:52.767793)', 9.93113528253442, 0, 65, 170, 'M', 25]
Data 2: ['mobile2', '12-Jun-2021 (19:44:52.767793)', 0, 0, 80, 185, 'M', 27]
Receiving data from url
Data 1: ['m

KeyboardInterrupt: 